In [ ]:
from bs4 import BeautifulSoup as bs
import pandas as pd
import requests
def cleanup_page(url):
    page = requests.get(url)
    soup = bs(page.content, 'html.parser')
    mc = soup.find('div', {'id': 'main-content'}).extract()
    for div in mc.find_all("a", {'class':'headerlink'}): 
        div.decompose()
    title = mc.find('h1').text
    for tag in mc.find_all(['div', 'span']): 
        tag.unwrap()
    for tag in mc.find_all("pre"): 
        tag.string = tag.text.replace('%%sql', '')
    for tag in mc.find_all(["script", 'h1']): 
        tag.decompose()
    mc.find('div')
    clean_html_filtered = filter(lambda e: str(e).strip() not in ('', '\n'), mc.contents)
    clean_html_list = [str(e) for e in clean_html_filtered]
    clean_html_list = filter(lambda e: str(e).strip() not in (''), str(bs('\n'.join(clean_html_list), 'html.parser')).splitlines())
    return title, str(bs('\n'.join(clean_html_list), 'html.parser'))

In [ ]:
from bs4 import BeautifulSoup as bs
import pandas as pd
import requests

def get_landing_page_url(base_url):

    base_page = requests.get(base_url)
    base_soup = bs(base_page.content, 'html.parser')

    landing_page_url = f"{base_url}/{base_soup.find('meta')['content'].split('=')[1]}"
    return landing_page_url

In [ ]:
def get_first_level_urls(landing_page_url):
    landing_page = requests.get(landing_page_url)
    landing_soup = bs(landing_page.content, 'html.parser')
    
    first_level_urls = []
    for link in landing_soup.find_all('a', {'class': 'reference internal'}):
        if link.get('href') and link.get('href') != '#':
            first_level_urls.append(link.get('href'))
    
    return first_level_urls

In [ ]:
def get_all_urls(first_level_urls):
    all_urls = []
    for url in first_level_urls:
        all_urls.append(f"{base_url}/{url}")
        second_level_url = f"{base_url}/{url}"
        second_level_page = requests.get(second_level_url)
        second_level_soup = bs(second_level_page.content, 'html.parser')
        current_active = second_level_soup.find('li', {'class': 'toctree-l1 current active'})
        for link in current_active.find_all('a', {'class': 'reference internal'}):
            all_urls.append(f"{base_url}/{url.split('/')[0]}/{link.get('href')}")
    return all_urls

In [ ]:
def scrape(base_url):
    landing_page_url = get_landing_page_url(base_url)
    first_level_urls = get_first_level_urls(landing_page_url)
    all_urls = get_all_urls(first_level_urls)
    return all_urls

In [ ]:
def check_yt(html):
    soup = bs(html, 'html.parser')
    for iframe in soup.find_all('iframe'):
        if iframe.get('src') and 'youtube.com' in iframe.get('src'):
            return iframe.get('src').split('/')[-1].split('?')[0]
    return

In [ ]:
base_url = 'https://python.itversity.com'

all_urls = scrape(base_url)

videos = []
for url in all_urls:
    title, clean_html = cleanup_page(url)
    # Capturing Video id to update the metadata (such as description) in YouTube
    video_id = check_yt(clean_html)
    if video_id:  
        videos.append((url, title, video_id, clean_html))

In [56]:
YT_DESCRIPTION_TEMPLATE = '''
{DESCRIPTION}

You can access complete content of Mastering Python by following this Playlist on YouTube - {COURSE_PLAYLIST}

You can access complete content related to this video by clicking on {CONTENT_LINK}

{SOCIAL_LINKS}

{HASH_TAGS}
'''

In [57]:
def get_description(html):
    soup = bs(html, 'html.parser')
    p = soup.find('p')
    return p.text

In [58]:
PYTHON_PLAYLIST = 'https://www.youtube.com/playlist?list=PLf0swTFhTI8rkH4yIfoyTAheEGjWIRtPG'

In [59]:
SOCIAL_LINKS = '''
* Newsletter: http://notifyme.itversity.com
* LinkedIn: https://www.linkedin.com/company/itversity/
* Facebook: https://www.facebook.com/itversity
* Twitter: https://twitter.com/itversity
* Instagram: https://www.instagram.com/itversity/
* YouTube: https://www.youtube.com/itversityin
'''

In [60]:
HASH_TAGS = '#MasteringPython #BuildApplications #DataEngineering #ExtremeAutomation'

In [61]:
videos_with_description = []
for video in videos:
    description = YT_DESCRIPTION_TEMPLATE.format(
        DESCRIPTION=get_description(video[3]),
        CONTENT_LINK=video[0],
        COURSE_PLAYLIST=PYTHON_PLAYLIST,
        SOCIAL_LINKS=SOCIAL_LINKS,
        HASH_TAGS=HASH_TAGS
    )
    videos_with_description.append((video[2], description))
    

In [ ]:
print(videos_with_description[0][1])